# Calculate and visualize lipid-protein distances interactively. APPLICATIONS HAVE NOT BEEN CONFIGURED YET.

In [1]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

Load data and build prolintpy objects

In [2]:
import mdtraj as md
import prolintpy as pl

In [4]:
t = md.load('./data/test_data_1.xtc', top='./data/test_data_1.gro')

In [5]:
resolution = "martini"
combine_proteins = False
lipids = pl.Lipids(t.topology, resolution=resolution)
proteins = pl.Proteins(t.topology, resolution=resolution).system_proteins(merge=combine_proteins)

In [8]:
contacts = pl.ComputeContacts(t, proteins, lipids)

# Calculate Distances 1

You can specify any residue list - lipid combination. Use the `percentile_co` and `distance_co` parameters to tweak the threshold when distances get registered and displayed. 
Setting `percentile_co` to zero will show any lipid that at least once crosses the `distance_co` cutoff. Increasing the `distance_co` will capture many more lipids. If you increase the `distance_co` too much 
and include many residues the application may become, understandably, unresponsive.<br>

Because some of the residues supplied may not satisfy the threshold criteria, it may happen that some of the residues in the application are empty (i.e. no distance is shown for them). This simply means 
that there was no contact that matched the provded threshold criteria. 

In [22]:
dist = contacts.compute_distances(t, proteins[0], [*range(85, 91)], 'POPS', 'PO4', percentile_co=0.05, distance_co=0.7)

In [24]:
pl.show_distances(dist)

Loading BokehJS ...

# Calculate distances 2

In [6]:
from prolintpy.core.computecontacts import retrieve_distances

We need to calculate contact-based metrics for this

In [25]:
cutoff = 0.5
result = contacts.compute_neighbors(t, cutoff=cutoff, grouped=False)

Working on protein copy: 0


In [27]:
df = pl.contacts_dataframe(result, proteins, t, radius=cutoff)

We first build a dictionary of lipids and residues that are interacting preferably and then we compute the distances between them. <br>
You can use the `top_nr` argument to indicate how many top ranking results you want to consider. 

This application is more computationally expensive since it requires the prior calculation of contact-based metrics, but it is much better in extracting good contacts. <br>
It also does not require any threshold definitions or manual input of lipid and residue ids. 

In [28]:
distances_dict, SYSTEM_LIPIDS, lipids_found = retrieve_distances(df, group_lipids=False, resolution=resolution, lipids=lipids, top_nr=30)
distances = contacts.compute_lipid_distances(t, proteins[0], distances_dict, SYSTEM_LIPIDS, lipids_found)

In [29]:
pl.show_metric_distances(distances)

Loading BokehJS ...